### Tests de connections à l'API OpenAI d'Oobabooga

In [3]:
!pip install openai

## Plan global du Notebook

1. **Introduction générale**  
   - Contexte et définitions (IA Générative, LLMs, prompts, etc.)
   - Brève présentation des enjeux éthiques et de la responsabilité

2. **Premier exemple de code : Vérification d’environnement**  
   - Installation/importation des bibliothèques (OpenAI ou Azure OpenAI)
   - Test d’un prompt de base

3. **Notions de base sur les prompts**  
   - Tokens, embeddings, bases de la génération
   - Exercices simples : tokenisation et génération

4. **Exemple : Fabrications (Hallucinations) et fiabilité**  
   - Démonstration d’un prompt volontairement ambigu
   - Discussion sur la vérification des faits

5. **Conclusion**  
   - Rappel des grandes idées
   - Activité suggérée (rédaction d’une synthèse ou extension)



# Introduction à l'IA Générative

Dans ce notebook, nous allons découvrir les bases de l'Intelligence Artificielle Générative. 

**Objectifs :**
- Comprendre ce qu’est l’IA Générative et en quoi elle consiste (texte, images, audio…)
- Découvrir le fonctionnement des modèles de langage de grande taille (LLMs)
- Mettre en pratique de premières expérimentations simples avec les prompts
- Aborder brièvement les questions de fiabilité (fabrications / “hallucinations”) et d’éthique

## Qu’est-ce que l’IA Générative ?

L’IA générative est une branche de l’apprentissage automatique qui **génère** de nouveaux contenus (texte, image, audio, code...) en se basant sur des modèles probabilistes entraînés sur de vastes quantités de données. Les modèles les plus avancés, souvent appelés **Large Language Models** (LLMs), utilisent des réseaux de neurones du type **Transformers** pour prédire et générer des séquences.

Exemples concrets :
- **ChatGPT** (OpenAI) : génération et compréhension de texte
- **Stable Diffusion** : génération d’images
- **Audiocraft**, **Whisper** : génération ou transcription audio
- **Copilot** (GitHub) : génération de code, complétion et explications

## Enjeux et limites
- **Hallucinations / Fabrications** : le modèle peut produire des réponses incorrectes ou inventées, même si elles semblent plausibles.
- **Biais** : ils peuvent refléter les biais présents dans les données d’entraînement (culture, genre, etc.).
- **Coût énergétique** : l’entraînement de grands modèles consomme beaucoup de ressources.
- **Régulation et éthique** : confidentialité, respect des lois, usage approprié de la technologie.

Nous allons maintenant mettre en place notre environnement et faire un premier test de prompt.

In [8]:
# ============================
# Cellule 2 : Configuration
# ============================

%pip install openai tiktoken python-dotenv 
import os
import openai
from dotenv import load_dotenv

# Charger la configuration depuis .env
load_dotenv()

# Récupérer la clé d'API
openai.api_key = os.getenv("OPENAI_API_KEY")

if openai.api_key is None:
    raise ValueError("Clé API OpenAI non trouvée. Vérifie ton .env !")

print("Clé API chargée avec succès !")


In [10]:
# ============================
# Cellule 3 : Premier prompt
# ============================

# Exemple simple : On veut générer une petite phrase.
# On utilise 'chat.completions.create' (nouvelle API).
# Le paramètre 'model' peut être par ex. gpt-3.5-turbo ou gpt-4 (ou un autre).

response = openai.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Oh say can you see"
        }
    ],
    model="gpt-4o-mini",  # ou gpt-4 si tu y as accès
    max_tokens=50,
    temperature=0.7
)

print("Réponse du modèle :")
print(response.choices[0].message.content)


# Notions de base : Tokenisation

Lorsque nous envoyons un prompt à un LLM, le texte est d’abord converti en une séquence de **tokens**. 
- Un token est généralement un morceau de mot, un caractère spécial, ou même un sous-mot.
- Plus le nombre de tokens est élevé, plus l’appel au modèle peut coûter cher en ressources (et parfois en argent, selon la facturation).

Dans les bibliothèques OpenAI, on peut utiliser la librairie `tiktoken` pour comprendre comment le modèle va découper le texte.

**Exercice** : Voyons comment le prompt « Oh say can you see » est découpé en tokens.


In [11]:
# ============================
# Cellule 5 : Tokenisation
# ============================

import tiktoken

# Choisir l'encodeur en fonction du modèle
# (ex: "gpt-3.5-turbo", "text-davinci-003", etc.)
encoder = tiktoken.encoding_for_model("text-davinci-003")

my_text = "Oh say can you see"

tokens = encoder.encode(my_text)
print("Liste des tokens (indexes) :\n", tokens)

decoded = [encoder.decode([t]) for t in tokens]
print("\nDécodage token par token :")
for i, t in enumerate(decoded):
    print(f"Token {i}: '{t}'")


# Exemple de fabrication (ou "hallucination")

Les modèles de langage peuvent parfois renvoyer des informations inventées ou inexactes. 
Testons un prompt ambigu ou factuellement faux pour voir comment le modèle réagit.

Par exemple : 

- Lui demander de décrire « la guerre de 2076 sur Mars » 
- Ou lui demander des détails d’une loi imaginaire

Ensuite, observer la réponse et la façon dont le modèle peut inventer des précisions ou admettre ne pas connaître l’information.


In [14]:
# ============================
# Cellule 7 : Fabrication
# ============================
prompt_fabrication = """
Décris-moi la célèbre guerre de 2076 sur la planète Mars :
- Qui étaient les grandes puissances en conflit ?
- Quels traités de paix ont été signés ?
"""

response_fabrication = openai.chat.completions.create(
    messages=[{"role": "user", "content": prompt_fabrication}],
    model="gpt-4o-mini",
    max_tokens=150,
    temperature=0.7
)

print("Réponse du modèle :\n")
print(response_fabrication.choices[0].message.content)


# Conclusion de cette introduction

Nous avons abordé :
- Les bases de l'IA générative et des LLMs
- Comment configurer et appeler rapidement un modèle (OpenAI)
- Les notions de tokenisation et d'impact sur la génération
- Un aperçu des “fabrications” qu’un modèle peut produire

## Pistes pour aller plus loin

- Expérimenter différentes **températures** (0.0, 0.7, etc.) pour influencer la créativité de la réponse.
- Utiliser l’API **Chat** (chat completions) plutôt que l’API `Completion` pour gérer des dialogues contextuels plus complexes.
- Mettre en place de la **RAG** (Retrieval Augmented Generation) pour limiter les hallucinations en donnant au modèle des sources documentaires externes.
- Tester la **traduction**, la **rédaction de code** (style Copilot) ou la génération de **contenu marketing**.

**Prochaines étapes** dans le cours :
1. Approfondir le **prompt engineering** (prompt structuré, chaîne d’invocations, etc.).
2. Explorer d’autres types de modèles génératifs (images, audio).
3. Étudier en détail les **dimensions éthiques** (biais, utilisation responsable, confidentialité).

---

Merci d'avoir suivi ce premier Notebook d'introduction !
